# Model derived from Mobilenet using transfer learning



In [46]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

GPU is NOT AVAILABLE


## Configuration

In [78]:
import ipywidgets as widgets

# MODEL_CHARACTERISTICS
MODELS = {
    "InceptionV1" : {
        "url": "https://www.kaggle.com/models/google/inception-v1/TensorFlow2/feature-vector/2",
        "dim": 224
    },
    "InceptionV2" : {
        "url": "https://www.kaggle.com/models/google/inception-v2/TensorFlow2/feature-vector/2",
        "dim": 224
    },
    "InceptionV3" : {
        "url": "https://www.kaggle.com/models/google/inception-v3/TensorFlow2/feature-vector/2",
        "dim": 299
    },
    "MobileNetV1" : {
        "url": "https://www.kaggle.com/models/google/mobilenet-v1/TensorFlow2/100-224-feature-vector/2",
        "dim": 224
    },
    "MobileNetV2" : {
        "url": "https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/100-224-feature-vector/2",
        "dim": 224
    },
    "MobileNetV3" : {
        "url": "https://www.kaggle.com/models/google/mobilenet-v3/TensorFlow2/small-100-224-feature-vector/1",
        "dim": 224
    },
    "NasNet" : {
        "url": "https://www.kaggle.com/models/google/nasnet/TensorFlow2/large-feature-vector/2",
        "dim": 331
    },
    "NasNetMobile" : {
        "url": "https://www.kaggle.com/models/google/nasnet/TensorFlow2/mobile-feature-vector/2",
        "dim": 224
    },
    "ResNetV1" : {
        "url": "https://www.kaggle.com/models/google/resnet-v1/TensorFlow2/50-feature-vector/2",
        "dim": 224
    },
    "ResNetV2" : {
        "url": "https://www.kaggle.com/models/google/resnet-v2/TensorFlow2/50-feature-vector/2",
        "dim": 224
    },
    
}
MODEL_CFG = widgets.Dropdown(description="Model:", value='InceptionV1', options=sorted(MODELS.keys()))
CT_CFG = widgets.FloatSlider(description="Convergenge threshold:", value=0.95, min=0.25, max=1.00, step=0.01)
DROPOUT_CFG = widgets.FloatSlider(description="Dropout (0: no dropout):", value=0.0, min=0.0, max=0.5, step=0.1)
MAX_STEPS_CFG = widgets.IntSlider(description="Max. training steps:", value=50, min=10, max=100, step=10)
BATCH_SIZE_CFG = widgets.IntSlider(description="Batch size:", value=168, min=32, max=300)
display(widgets.VBox([MODEL_CFG, CT_CFG, DROPOUT_CFG, MAX_STEPS_CFG, BATCH_SIZE_CFG]))

In [90]:
P_MODEL = MODEL_CFG.value
P_DIM = MODELS[P_MODEL]["dim"]
P_URL = MODELS[P_MODEL]["url"]
global P_CT
P_CT = CT_CFG.value
P_DROPOUT = DROPOUT_CFG.value
P_MAX_STEPS = MAX_STEPS_CFG.value
P_BATCH_SIZE = BATCH_SIZE_CFG.value
print(' Model:', P_MODEL, '\n',
      'Model - URL:', P_URL, '\n',
      'Model - dim:', P_DIM, '\n',
      'Conv. thr.:', P_CT, '\n',
      'Dropout:', P_DROPOUT, '\n',
      'Max steps:', P_MAX_STEPS, '\n',
      'Batch size:', P_BATCH_SIZE)

 Model: MobileNetV2 
 Model - URL: https://www.kaggle.com/models/google/mobilenet-v2/TensorFlow2/100-224-feature-vector/2 
 Model - dim: 224 
 Conv. thr.: 0.95 
 Dropout: 0.1 
 Max steps: 50 
 Batch size: 168


## Define train, validation and test sets

In [91]:
datagen = ImageDataGenerator(rescale=1.0/255.0)
train_dataset = datagen.flow_from_directory(
    'dset/train',
    target_size=(P_DIM,P_DIM),
    batch_size=P_BATCH_SIZE,
    class_mode='sparse'
)
validation_dataset = datagen.flow_from_directory(
    'dset/valid',
    target_size=(P_DIM,P_DIM),
    batch_size=P_BATCH_SIZE,
    class_mode='sparse'
)
test_dataset = datagen.flow_from_directory(
    'dset/test',
    target_size=(P_DIM,P_DIM),
    batch_size=P_BATCH_SIZE,
    class_mode='sparse'
)

Found 1680 images belonging to 21 classes.
Found 210 images belonging to 21 classes.
Found 210 images belonging to 21 classes.



## Define CNN using Keras API


In [92]:
kl = hub.KerasLayer(P_URL, trainable=False, input_shape=(P_DIM, P_DIM, 3))
fl = tf.keras.layers.Dense(21, activation='softmax')

if P_DROPOUT > 0.0:
    model = tf.keras.Sequential([ kl, tf.keras.layers.Dropout(P_DROPOUT), fl])
else:
    model = tf.keras.Sequential([ kl, fl])

model.summary()

URLError: <urlopen error [Errno -3] Temporary failure in name resolution>

## Train model

In [ ]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

class CustomCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        global P_CT
        if logs.get('accuracy') >= P_CT:
            self.model.stop_training = True

history = model.fit(
     train_dataset,
     epochs=P_MAX_STEPS,
     validation_data=validation_dataset,
     callbacks = [CustomCallback()]
)

## Plot training evolution

In [ ]:
from  matplotlib import pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


## Save model and convert it to TFLite format

In [ ]:
model.save('models/' + P_MODEL)
converter = tf.lite.TFLiteConverter.from_saved_model('models/' + P_MODEL) # path to the SavedModel directory
tflite_model = converter.convert()
with open('models/' + P_MODEL + '.tflite', 'wb') as f:
      f.write(tflite_model)

## Calculate accuracy and loss for validation and test datasets

In [ ]:
loss, accuracy = model.evaluate(validation_dataset)

print("VALIDATION - loss {}, accuracy {}".format(loss,accuracy))

loss, accuracy = model.evaluate(test_dataset)

print("TEST - loss {}, accuracy {}".format(loss,accuracy))